In [ ]:
import cv2
import numpy as np
import h5py
import os
import re
from tqdm import tqdm
import pandas as pd
from matplotlib import pyplot as plt
import sys
import joblib


In [ ]:
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
IMAGE_FILES = []
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [ ]:
data_folder_path = '/xgaze_224/train/'

save_folder_right = ''
save_folder_left = ''
save_csv_folder = ''

os.makedirs(save_folder_right, exist_ok=True)
os.makedirs(save_folder_left, exist_ok=True)
os.makedirs(save_csv_folder, exist_ok=True)

right_list = [7,33,112,130,133,144,155,153,154,157,158,159,160,160,161,162,163, 173]
left_list = [249,255,341,359,382,373,374,380,381,381,384,385,386,387,388,390,398,466]
# validation_subject = ['subject0010','subject0020','subject0030','subject0040','subject0050','subject0060','subject0070','subject0080','subject0090','subject0100','subject0110']

In [ ]:
import math
from mediapipe.framework.formats import landmark_pb2
from typing import List, Mapping, Optional, Tuple, Union
landmark_pb2.NormalizedLandmarkList
import numpy as np

_PRESENCE_THRESHOLD = 0.5
_VISIBILITY_THRESHOLD = 0.5
_BGR_CHANNELS = 3

ldmks_list = []


def _normalized_to_pixel_coordinates(
    normalized_x: float, normalized_y: float, image_width: int,
    image_height: int) -> Union[None, Tuple[int, int]]:
  """Converts normalized value pair to pixel coordinates."""

  # Checks if the float value is between 0 and 1.
  def is_valid_normalized_value(value: float) -> bool:
    return (value > 0 or math.isclose(0, value)) and (value < 1 or
                                                      math.isclose(1, value))

  if not (is_valid_normalized_value(normalized_x) and
          is_valid_normalized_value(normalized_y)):
    # TODO: Draw coordinates even if it's outside of the image bounds.
    return None
  x_px = min(math.floor(normalized_x * image_width), image_width - 1)
  y_px = min(math.floor(normalized_y * image_height), image_height - 1)
  return x_px, y_px

In [ ]:
def gaze_calc_show(gaze_data):
    global img_num
    alpha = gaze_data[0]
    beta = gaze_data[1]
    gt_gaze_x = -1 * np.sin(beta) * np.cos(alpha)
    gt_gaze_y = -1 * np.sin(alpha)
    gt_gaze_z = -1 * np.cos(alpha) * np.cos(beta)
    
    return [gt_gaze_x, gt_gaze_y, gt_gaze_z]

def gaze_calc(gaze_data, path):
    global img_num
    alpha = gaze_data[0]
    beta = gaze_data[1]
    gt_gaze_x = -1 * np.sin(beta) * np.cos(alpha)
    gt_gaze_y = -1 * np.sin(alpha)
    gt_gaze_z = -1 * np.cos(alpha) * np.cos(beta)
    
    return [path, gt_gaze_x, gt_gaze_y, gt_gaze_z]

In [ ]:
def show_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

def show_gaze_img(img, gaze):
    x = int(img.shape[1]/2)
    y = int(img.shape[0]/2)
    img = cv2.circle(img, (x, y), 1, (0,0,255), thickness = 1)
    img = cv2.arrowedLine(img, (x, y), (x + int(50*gaze[0]), y + int(50*gaze[1])), (255, 0, 0), thickness=1)
    show_img(img)
    

In [ ]:
subject_list = sorted(os.listdir(data_folder_path), key=lambda s: int(re.search(r'\d+', s).group()))
# print(subject_list)
img_num = 1
gaze_datas = []
all_img_name_cnt = 1

In [ ]:
def img_cropper(img, ldmk_x, ldmk_y):
    if ldmk_y - 18 < 0:
        movement_h = int(round(18 - ldmk_y, 0))
    elif ldmk_y + 18 > img.shape[0]:
        movement_h = int(img.shape[0] - (ldmk_y + 18))
    else:
        movement_h = 0

    if ldmk_x - 30 < 0:
        movement_w = int(round(30 - ldmk_x, 0))
    elif ldmk_x + 30 > img.shape[1]:
        movement_w = int(round(img.shape[1] - (ldmk_x + 30)))
    else:
        movement_w = 0
    
    return img[int(round(ldmk_y-18, 0)+movement_h):int(round(ldmk_y+18, 0)+movement_h), int(round(ldmk_x-30, 0))+movement_w:int(round(ldmk_x+30,0))+movement_w]
    

In [ ]:
subject_list = sorted(subject_list)

for subject in subject_list:
    subject_folder_path_left = os.path.join(save_folder_left, subject[7:-3])
    subject_folder_path_right = os.path.join(save_folder_right, subject[7:-3])
    os.makedirs(subject_folder_path_left, exist_ok=True)
    os.makedirs(subject_folder_path_right, exist_ok=True)

In [ ]:
for subject_data in subject_list:
    csv_name = os.path.join(save_csv_folder, subject_data[7:-3] + '.csv')
    print(csv_name)

In [ ]:
def split_eth_eval_data(subject_data):

    all_img_name_cnt = 1
    gaze_datas = []
    
    global save_folder_right
    global save_folder_left
    global data_folder_path
    global save_csv_folder

    csv_name = os.path.join(save_csv_folder, subject_data[7:-3] + '.csv')
    read_data_path = os.path.join(data_folder_path, subject_data)
    fid = h5py.File(read_data_path, 'r')
    img_size = 200
    # img_show = np.zeros((img_size*3, img_size*6, 3), dtype=np.uint8)  # initial a empty image
    gaze = []

    for num_i in tqdm(range(fid["face_patch"].shape[0])):
        face_patch = fid['face_patch'][num_i]  # the face patch
        # print(subject_data+' '+str(num_i))

        if 'face_gaze' in fid.keys():
            gaze = fid['face_gaze'][num_i]
            # gaze = gaze_calc_show(gaze)
            fid_path = read_data_path + str(num_i)
            gaze_datas.append(gaze_calc(gaze, fid_path))

            face_patch = cv2.resize(face_patch, (img_size, img_size)) #250x250
            gray = cv2.cvtColor(face_patch, cv2.COLOR_BGR2GRAY)

            right_ldmks = []
            left_ldmks = []
            ldmks_list = []

            with mp_face_mesh.FaceMesh(
                static_image_mode=True,
                max_num_faces=1,
                refine_landmarks=True,
                min_detection_confidence=0.5) as face_mesh:

                image = face_patch
                image_rows, image_cols, _ = image.shape
                results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                annotated_image = image.copy()

                if results.multi_face_landmarks == None:

                    right_ldmks_center = [60,90]
                    left_ldmks_center = [130,90]
                    save_img_right = img_cropper(gray, right_ldmks_center[0], right_ldmks_center[1])
                    save_img_left = img_cropper(gray, left_ldmks_center[0], left_ldmks_center[1])

                    save_img_name = str(all_img_name_cnt) + '.png'
                    if save_img_right.shape[0] == 36 and save_img_right.shape[1] == 60:
                        save_img_right_path = os.path.join(save_folder_right, subject_data[7:-3],  save_img_name)
                        cv2.imwrite(save_img_right_path, save_img_right)

                    else:
                        sys.exit('Error Right Image')

                    if save_img_left.shape[0] == 36 and save_img_left.shape[1] == 60:
                        save_img_left_path = os.path.join(save_folder_left, subject_data[7:-3], save_img_name)
                        cv2.imwrite(save_img_left_path, save_img_left)

                    else:
                        sys.exit('Error Left Image')

                    all_img_name_cnt += 1

                else:

                    for face_landmarks in results.multi_face_landmarks:
                        mp_drawing.draw_landmarks(
                        image=annotated_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_tesselation_style())

                    for idx, landmark in enumerate(face_landmarks.landmark):
                        if ((landmark.HasField('visibility') and
                            landmark.visibility < _VISIBILITY_THRESHOLD) or 
                            (landmark.HasField('presence') and
                            landmark.presence < _PRESENCE_THRESHOLD)):
                            continue
                        landmark_px = _normalized_to_pixel_coordinates(landmark.x, landmark.y,
                                                                    image_cols, image_rows)

                        if landmark_px is not None:
                            ldmks_list.append(landmark_px)
                        else:
                            zero_coord = (0,0)
                            ldmks_list.append(zero_coord)                                                
                        # print(landmark_px)
                        
                    ldmks_array = np.array(ldmks_list)
                    # print(ldmks_array.shape)

                    if len(ldmks_array.shape) == 1:
                        for r_num in right_list:
                            if ldmks_array[r_num] != None:
                                right_ldmks.append(ldmks_array[r_num])
                        for l_num in left_list:
                            if ldmks_array[l_num] != None:
                                left_ldmks.append(ldmks_array[l_num])
                    
                    else:
                        for r_num in right_list:
                            if ldmks_array[r_num][0] != None:
                                right_ldmks.append(ldmks_array[r_num])
                        for l_num in left_list:
                            if ldmks_array[l_num][0] != None:
                                left_ldmks.append(ldmks_array[l_num])

                    right_ldmks_center = np.mean(right_ldmks, axis=0)
                    left_ldmks_center = np.mean(left_ldmks, axis=0)

                    save_img_right = img_cropper(gray, right_ldmks_center[0], right_ldmks_center[1])
                    save_img_left = img_cropper(gray, left_ldmks_center[0], left_ldmks_center[1])


                    save_img_name = str(all_img_name_cnt) + '.png'
                    if save_img_right.shape[0] == 36 and save_img_right.shape[1] == 60:
                        save_img_right_path = os.path.join(save_folder_right, subject_data[7:-3], save_img_name)
                        cv2.imwrite(save_img_right_path, save_img_right)

                    else:
                        sys.exit('Error Right Image')

                    if save_img_left.shape[0] == 36 and save_img_left.shape[1] == 60:
                        save_img_left_path = os.path.join(save_folder_left,subject_data[7:-3], save_img_name)
                        cv2.imwrite(save_img_left_path, save_img_left)

                    else:
                        sys.exit('Error Left Image')

                    all_img_name_cnt += 1

    df_gaze = pd.DataFrame(gaze_datas)
    df_gaze.to_csv(csv_name, index=False,header = False)

In [ ]:
subject_list = sorted(os.listdir(data_folder_path), key=lambda s: int(re.search(r'\d+', s).group()))

result = joblib.Parallel(n_jobs=-1,backend='threading')(joblib.delayed(split_eth_eval_data)(i) for i in subject_list)
# result = joblib.Parallel(n_jobs=4)(joblib.delayed(split_eth_eval_data)(i) for i in subject_list)